# Registration of confocal images of Drosophila abdomens 

This notebook runs the pipeline to register the confocal images of fly abdomens.
The analysis is divided in few steps.
Each step uses functions saved in files in the src folder and its subfolders. 

The results of each step of the analysis are saved in a series of numbered subfolders in the data folder.

The input raw data should be saved in the 01_raw subfolder as separate .tif files for each channel with a prefix that identifies the channel (like C1-filename.tif).  It should also contain an excel file with the columns "image file name", "type" and "quality". These columns contain the name of each image stack, a label identifying to which group or genotype the image stack belongs to, and a label characterizing the quality of the raw data for filtering the results at a later point in the analysis.


In [1]:
# import packaging and add the src folder to the path.

import os
import sys
import pandas as pd

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

## 1. Automated segmentation of the abdomens

In [2]:
# Importing the preprocessing function from the preprocess.py file:

from src.preprocess import preprocess_and_segment_images

raw_data_folder = "../test_dataset/01_raw"
destination_folder = "../test_dataset/02_preprocessed"

#downscaling of confocal stacks along z,x,y to make the resolution isotropic:
downscaling = (1, 2.5, 2.5) 

# bit depth of raw images (depending on the detector settings may be 8, 12 or 16)
bit_depth = 12

# name of the excel file containing the list of images to be used in the analysis. 
db_name = 'DatasetInformation.xlsx'

# default values of the parameters used for the segmentation:
segm_pars={'threshold': 1.05, 'max_iter': 200,'fraction_range': [0.04, 0.05], 
           'padding': 20, 'closing_r': 4, 'dilation_r': 3, 'mesh_radius': 10}

preprocess_and_segment_images(raw_data_folder, destination_folder, downscaling, bit_depth, 
                              only_on_new_files = False, segmentation_parameters=segm_pars, 
                              database_filename = db_name)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Preprocessing of raw images in progress:


100%|█████████████████████████████████████████████| 3/3 [00:06<00:00,  2.06s/it]


## 2. Manual registration of the segmented 3D images


In [3]:
# Importing the registration function from the registration.py file:

from src.registration import run_manual_registration

"""
# USER INPUT IS REQUIRED, two windows with a 3D rendering of the detected object
will pop-up for each image in the dataset. 
The first window shows the source object to register, the second one shows the reference target object.

For each couple of images:
1)  Please pick at least three correspondences using [shift + left click]
    Press [shift + right click] to undo point picking)
2)  After picking points, press 'Q' to close the window
3)  Repeat points 1 and 2 for the reference image in the second window.
4)  The next window will show the result of the registration, press 'Q' to close the window.

"""

read_folder = "../test_dataset/02_preprocessed"
destination_folder = "../test_dataset/03_registered"

reference_fly_filename = "../test_dataset/References_and_masks/C1_Reference_iso.tiff"

df_name = "DatasetInformation.xlsx"

preprocessed_df = pd.read_excel(os.path.join(read_folder,df_name))
run_manual_registration(preprocessed_df, read_folder, reference_fly_filename, 
                        destination_folder, only_on_new_files = True)

Registration of 3D stacks in progress:


100%|████████████████████████████████████████████| 3/3 [09:11<00:00, 183.78s/it]


## 3. Automated 2D projection of registered 3D images 

In [3]:
from src.projection import run_2D_projection

read_folder = "../test_dataset/03_registered"
destination_folder = "../test_dataset/04_projected"
landmark_folder = "../test_dataset/05_landmarks/data"
abdomen_mask_file = "../test_dataset/References_and_masks/Reference_mask_iso_thick.tiff"
abdomen_shape_reference_file = "../test_dataset/References_and_masks/Reference_mask_iso.tiff"

df_name = "DatasetInformation.xlsx"
df = pd.read_excel(os.path.join(read_folder,df_name))

# default values used by the projection algorithm:
params = {'min_y':0, 'meridian_plane_x':62,'spline_smoothing':5, 'projection_radius':5}

run_2D_projection(df, read_folder, destination_folder, landmark_folder, abdomen_mask_file,
                  abdomen_shape_reference_file, crop_x=150, crop_y = 150, only_on_new_files = False, 
                  projection_parameters = params)

Projection of registered 3D stacks to 2D images in progress:


100%|█████████████████████████████████████████████| 3/3 [00:06<00:00,  2.17s/it]


##  4. Annotation and warping of the 2D projected images

After the projection from 3D to 2D, images are already roughly aligned. However, the alignment up to this point was only performed by applying a global rescaling and rigid rotations/translations of the original stacks.

To fine tune the registration we need to perform an elastic transformation of the final images. The following cell of the notebook starts a graphical interface that allows to manually annotate landmarks on the projected images and then register them on a reference.

in the interface, open the annotation project, add the new images found in the folder 05_landmarks/data to the project. Annotate the new images and run the registration, save the results in the folder 06_warped.

In [2]:
from src.image_registration_gui import gui

gui.start(main_window_size = (1000, 1000), graph_canvas_width = 600)

## 5. Final touch: smoothing and masking

The final step applies a binary mask and a gaussian smoothing filter to the images. The width of the gaussian smoothing can be set to different values along the x and y axis.

In [4]:
import pandas as pd
from src.masking_smoothing import run_smoothing_masking

destination_folder = "../test_dataset/07_masked_and_smooth"
read_folder = "../test_dataset/06_warped"
mask = '../test_dataset/References_and_masks/Mask2d.tif'
database_registered_images = "../test_dataset/06_warped/dataframe_info.csv"
database_info = "../test_dataset/04_projected/DatasetInformation.xlsx"

df_images = pd.read_csv(database_registered_images)
df_info = pd.read_excel(database_info)

smooth_x = 2
smooth_y = 1

run_smoothing_masking(
        read_folder, destination_folder, df_images, df_info,
        mask, smooth_x, smooth_y, bcg_types=['empty'], bcg_channels=['C2'], refine_mask = False, binning=None)

/media/ceolin/Data/Lab Gompel/Projects/Fly_Abdomens/abdomens_registration/notebooks/../src/masking_smoothing.py:81: RuntimeWarning: divide by zero encountered in true_divide
  norm_mask_tmp = (1 / aux_convolution(
/media/ceolin/Data/Lab Gompel/Projects/Fly_Abdomens/abdomens_registration/notebooks/../src/masking_smoothing.py:81: RuntimeWarning: invalid value encountered in multiply
  norm_mask_tmp = (1 / aux_convolution(


Smoothing and masking of the registered images in progress:


100%|████████████████████████████████████████████| 9/9 [00:00<00:00, 223.11it/s]
